## 5.1 构建神经网络

神经网络由执行数据操作的层、模块组成。

`torch.nn` 命名空间提供了构建自己的神经网络所需的所有构建块。

PyTorch 中的每个模块都是 `nn.Module` 的子类。

神经网络本身就是一个模块，由其他模块（层）组成。这种嵌套结构使得构建和管理复杂的体系结构变得容易。

在接下来的章节中，我们将构建一个神经网络，用于对 FashionMNIST 数据集中的图像进行分类。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 5.2 获取训练设备

我们希望能够在硬件加速器上，如GPU或MPS上训练我们的模型，如果可用的话。

让我们来检查一下 `torch.cuda` 或 `torch.backends.mps` 是否可用，否则我们就使用CPU。

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 5.3 定义类

我们通过子类化 `nn.Module` 来定义神经网络，并在 `__init__` 中初始化神经网络层。

每个 `nn.Module` 子类都在 `forward` 方法中实现对输入数据的操作。

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建了一个 `NeuralNetwork` 实例，并将其移动到 `device` 中，并打印出其结构。

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用这个模型，我们需要将输入数据传递给它。这将执行模型的`前向传播`，同时进行一些`反向传播`。请勿直接调用 `model.forward()` !

将输入数据传递给模型后，会返回一个二维张量。

其中 dim=0 对应于每个类别的 10 个原始预测值的输出，而 dim=1 对应于每个输出的各个值。

我们可以通过将其通过 `nn.Softmax` 模块的实例来得到预测概率。

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


## 5.4 模型层

让我们来分解一下FashionMNIST模型中的层。

为了说明这一点，我们将取一个大小为 28x28 的样本小批量，其中包含 3 张图像，并观察当我们将它们通过网络传递时会发生什么。

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### 5.4.1 nn.Flatten

我们初始化 `nn.Flatten` 层，

将每个 2D 的 28 x 28 的图像转换为一个连续的包含 784 个像素值的数组。

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### 5.4.2 nn.Linear

`linear layer`是一个模块，它使用其存储的权重和偏置对输入进行线性变换。


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### 5.4.3 nn.ReLU

非线性激活函数是创建模型输入和输出之间复杂映射的关键。它们在线性变换之后应用，引入了非线性，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用 `nn.ReLU`，但是还有其他激活函数可以在您的模型中引入非线性。

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2796, -0.0885, -0.2289,  0.0924, -0.2472, -0.6480,  0.2824,  0.6033,
          0.0321,  0.1214,  0.1462, -0.3680,  0.2257, -0.5077, -0.0540,  0.2447,
         -0.8236, -0.3341,  0.1597,  0.0937],
        [ 0.4531,  0.1069,  0.1352,  0.2002, -0.4846, -0.4914,  0.3271,  0.6315,
         -0.0546, -0.3370,  0.0453, -0.3834,  0.1811, -0.5864, -0.3483,  0.5704,
         -0.2976,  0.0353,  0.0304,  0.0782],
        [ 0.1446, -0.1517, -0.2952,  0.1683, -0.4407, -0.2303,  0.7073,  0.5604,
         -0.0365,  0.0849,  0.2138, -0.3378,  0.0824, -0.4955, -0.3556,  0.4089,
         -0.0563, -0.1849, -0.0629, -0.0628]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2796, 0.0000, 0.0000, 0.0924, 0.0000, 0.0000, 0.2824, 0.6033, 0.0321,
         0.1214, 0.1462, 0.0000, 0.2257, 0.0000, 0.0000, 0.2447, 0.0000, 0.0000,
         0.1597, 0.0937],
        [0.4531, 0.1069, 0.1352, 0.2002, 0.0000, 0.0000, 0.3271, 0.6315, 0.0000,
         0.0000, 0.0453, 0.0000, 0.1811, 0.0000, 0.00

### 5.4.4 nn.Sequential

`nn.Sequential` 是一个有序的模块容器。数据会按照定义的顺序依次通过所有的模块。你可以使用序列容器来快速组合一个网络，比如 `seq_modules`。

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[-0.0674, -0.2465,  0.0110, -0.4731,  0.2981,  0.1497,  0.0888,  0.2502,
          0.0268,  0.2909],
        [ 0.0010, -0.2147,  0.0562, -0.4191,  0.1908,  0.2029,  0.0864,  0.2431,
          0.0937,  0.3822],
        [-0.1288, -0.2415,  0.0126, -0.4551,  0.2465,  0.1561, -0.0112,  0.2419,
         -0.0270,  0.2001]], grad_fn=<AddmmBackward0>)

### 5.4.5 nn.Softmax

神经网络的最后一层线性层返回 `logits`，这些值被传递到 `nn.Softmax` 模块中。这些 logits 被缩放到值为 [0, 1] 的概率表示，代表模型对每个类别的预测概率。`dim` 参数指定了哪个维度上的值必须总和为 1。

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0882, 0.0737, 0.0954, 0.0588, 0.1271, 0.1096, 0.1031, 0.1211, 0.0969,
         0.1262],
        [0.0919, 0.0741, 0.0972, 0.0604, 0.1112, 0.1125, 0.1001, 0.1171, 0.1009,
         0.1346],
        [0.0860, 0.0769, 0.0991, 0.0621, 0.1252, 0.1144, 0.0968, 0.1247, 0.0953,
         0.1196]], grad_fn=<SoftmaxBackward0>)

## 5.5 模型参数

神经网络中的许多层都是*参数化*的，即具有相关的权重和偏置项，这些参数在训练过程中被优化。子类化 `nn.Module` 会自动跟踪模型对象中定义的所有字段，并使所有参数可以使用模型的 `parameters()` 或 `named_parameters()` 方法进行访问。

在此示例中，我们迭代每个参数，并打印其大小和值的预览。

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0268,  0.0090, -0.0211,  ...,  0.0191,  0.0018,  0.0057],
        [ 0.0079, -0.0317,  0.0205,  ..., -0.0222,  0.0309, -0.0287]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0031, -0.0225], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0131, -0.0195, -0.0282,  ..., -0.0139,  0.0105, -0.0189],
        [ 0.0210, -0.0186,  0.0237,  ..., -0.0392, -0.0377, -0.0329]],
       device='cuda:0', grad_fn=<Sl